In [15]:
import pandas as pd
import numpy as np


# encodings_to_try = ["utf-8", "latin-1", "ISO-8859-1", "cp1252"]
files = [
    "./sampleQuestionFB_Angel - sampleQuestionFB.csv",
    "./sampleQuestionFB_Ruben - sampleQuestionFB.csv",
    "./sampleQuestionFB_Amir - sampleQuestionFB.csv",
]


dflst = []
for file in files:
    no1 = pd.read_csv(file, encoding="utf-8")
    no1.iloc[:, 0] = (
        no1.iloc[:, 0]
        .dropna()
        .str.encode("ascii", "ignore")
        .str.decode("ascii")
        .str.strip()
        .str.lower()
    )
    dflst.append(no1)


dfQ = pd.concat(dflst).replace(2, 0).replace(3, 0).drop("No category", axis=1)
dfsum = dfQ.fillna(0).groupby("interest").sum().reset_index()
dfsum["validLst"] = dfsum.apply(
    lambda x: [i for i in dfQ.columns if (x[i] == 2) or (x[i] == 3)], axis=1
)
dfsum["validLst"] = dfsum["validLst"].apply(lambda x: x if len(x) > 0 else np.nan)
dfsum2 = dfsum[["interest", "validLst"]].dropna()
dfIg = pd.read_csv("sampleIg.csv", encoding="utf-8")[["IgName", "IgLevel2"]]
dfIg["IgName"] = (
    dfIg["IgName"]
    .dropna()
    .str.encode("ascii", "ignore")
    .str.decode("ascii")
    .str.strip()
    .str.lower()
)
dfagg = dfsum2.merge(dfIg, left_on="interest", right_on="IgName")
dfagg["validCat"] = dfagg.apply(lambda x: x["IgLevel2"] in x["validLst"], axis=1)
dfagg[dfagg["validCat"]].shape[0] * 100 / dfagg.shape[0]

71.95121951219512

In [16]:
dfsum2.shape[0]

82

In [17]:
files = [
    "./sampleQuestionIAB_Angel - sampleQuestionIAB.csv",
    "./sampleQuestionIAB_Ruben - sampleQuestionIAB.csv",
    "./sampleQuestionIAB_Amir - sampleQuestionIAB.csv",
]


dflst = []
for file in files:
    no1 = pd.read_csv(file, encoding="utf-8")
    no1.iloc[:, 0] = (
        no1.iloc[:, 0]
        .dropna()
        .str.encode("ascii", "ignore")
        .str.decode("ascii")
        .str.strip()
        .str.lower()
    )
    dflst.append(no1)


dfQ = pd.concat(dflst).replace(2, 0).replace(3, 0).drop("No category", axis=1)
dfsum = dfQ.fillna(0).groupby("interest").sum().reset_index()
dfsum["validLst"] = dfsum.apply(
    lambda x: [i for i in dfQ.columns if (x[i] == 2) or (x[i] == 3)], axis=1
)
dfsum["validLst"] = dfsum["validLst"].apply(lambda x: x if len(x) > 0 else np.nan)
dfsum2 = dfsum[["interest", "validLst"]].dropna()
dfIg = pd.read_csv("sampleGPT.csv", encoding="utf-8")[["interest", "iabCat"]]

dfIg["interest"] = (
    dfIg["interest"]
    .dropna()
    .str.encode("ascii", "ignore")
    .str.decode("ascii")
    .str.strip()
    .str.lower()
)
dfagg = dfsum2.merge(dfIg, left_on="interest", right_on="interest")
dfagg["validCat"] = dfagg.apply(lambda x: x["iabCat"] in x["validLst"], axis=1)
dfagg[dfagg["validCat"]].shape[0] * 100 / dfagg.shape[0]

79.01234567901234

In [18]:
dfsum2.shape[0]

80

In [45]:
df_gpt = pd.read_csv("./sampleGPT.csv")
gpt_df2 = (
    pd.get_dummies(df_gpt, columns=["iabCat"], prefix="")
    .replace(True, 1)
    .replace(False, 0)
)
gpt_df2.columns = gpt_df2.columns.str.replace("_", "")
gpt_df2.columns = gpt_df2.columns.str.replace("interest", "Topic")
gpt_df2.iloc[:, 0] = (
    gpt_df2.iloc[:, 0]
    .dropna()
    .str.encode("ascii", "ignore")
    .str.decode("ascii")
    .str.strip()
    .str.lower()
)
groundT = pd.concat([res, gpt_df2])
groundT

,Topic,Food & Drink,Automotive,No category,validResp
0,abarth,0,3.0,0,True
1,acura,0,3.0,0,True
2,airbag,0,3.0,0,True
3,airbus a380,0,3.0,0,True
4,alfajor,3.0,0,0,True
...,...,...,...,...,...
495,water,1,0,NaN,NaN
496,smart (automobile),0,1,NaN,NaN
497,toyota rav4,0,1,NaN,NaN
498,toyota tundra,0,1,NaN,NaN


In [46]:
groundT2 = groundT.groupby("Topic").sum()
groundT2["ValidPrediction"] = groundT2.apply(
    lambda x: x["Automotive"] == 4 or x["Food & Drink"] == 4, axis=1
)
groundT2
# groundT[groundT["No category"]==3]

,Food & Drink,Automotive,No category,validResp,ValidPrediction
Topic,,,,,
3d printing,0,1,0,0,False
abarth,0,4.0,0,True,True
abeokuta,0,1,0,0,False
acura,0,4.0,0,True,True
airbag,0,4.0,0,True,True
...,...,...,...,...,...
whisky,4.0,0,0,True,True
white chocolate,4.0,0,0,True,True
wine tasting,4.0,0,0,True,True


In [50]:
res.shape[0]

449

In [48]:
groundT2[groundT2["ValidPrediction"]].shape[0]

441

In [52]:
groundT2[groundT2["ValidPrediction"]].shape[0] / res.shape[0]

0.9821826280623608